<a href="https://colab.research.google.com/github/Pablobarchiesi94/PI_1_MLOps/blob/main/ETL%20/%20Limpieza_movies_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo las librerias que voy a utilizar

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import ast
import json

Me conecto con Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargo el Dataset

In [4]:
ruta_archivo = '/content/drive/MyDrive/movies_dataset.csv'
df= pd.read_csv(ruta_archivo)

<ipython-input-4-6743fc268ec4>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(ruta_archivo)


In [5]:
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


# **Limpieza de datos**

1. Elimino las columnas que me piden en las especificaciones del proyecto

In [6]:
columnas_descartables = ['video','imdb_id','adult','original_title','poster_path','homepage']
df = df.drop(columns=columnas_descartables)

2. Los valores nulos de los campos revenue y budget se rellenan con ceros.

In [7]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

3. Creo la columna release_year con el formato aaaa-mm-dd

In [8]:
# Borro los valores nulos

df = df.dropna(subset=['release_date'])

In [9]:
# Convierto a formato datatime

df['release_date'] = pd.to_datetime(df['release_date'],format='%Y-%m-%d',errors='coerce')

In [10]:
# Compruebo si tiene valores nulos

df['release_date'].isna().sum()

3

In [11]:
# Borro los valores nulos

df = df.dropna(subset=['release_date'])

In [12]:
# Compruebo si hay valores nulo

df['release_date'].isna().sum()

0

In [13]:
# Creó la columna release_year con formato datatime

df['release_year'] = df['release_date'].dt.year

4. Crear la columna return(retorno de inversion), dividiendo los campos revenue/budget, cuando no hay datos disponibles para calcular, debera tomar el valor 0.

In [14]:
# Paso de dato type object a float

df['budget'] = pd.to_numeric(df['budget'], downcast='float', errors='coerce')

In [15]:
# Divido los campos revenue/budget

df['return'] = df['revenue'].divide(df['budget']).fillna(0)

In [16]:
# Cambio los valores infinito por NAN

df['return'].replace([np.inf,-np.inf], np.nan, inplace=True)

In [17]:
# Cambio los NAN por cero

df['return'].fillna(0, inplace = True)

5. Algunos campos estan anidados deberan ser desanidados para unirlos al dataset.


In [18]:
# Desanido listas

def fetch_name(obj):
    if isinstance(obj,str) and '{' in obj:
        s = []
        for i in ast.literal_eval(obj):
            s.append(i['name']);
        return s

In [19]:
# Desanido diccionarios

def fetch_name1(obj):
    if isinstance(obj,str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [20]:
df['genres'] = df['genres'].apply(fetch_name)
df['belongs_to_collection'] = df['belongs_to_collection'].apply(fetch_name1)
df['production_companies'] = df['production_companies'].apply(fetch_name)
df['production_countries'] = df['production_countries'].apply(fetch_name)
df['spoken_languages'] = df['spoken_languages'].apply(fetch_name)

In [27]:
# Dataset con datos limpios

df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,DiaSemana,Mes
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,Lunes,Octubre
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,Viernes,Diciembre
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,Viernes,Diciembre
3,None,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,Viernes,Diciembre
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,Viernes,Febrero


Agrego una nueva columna 'DiaSemana' que contiene el día de la semana correspondiente

In [22]:
df['DiaSemana'] = df['release_date'].dt.day_name()

Reemplazo los nombres de los días en inglés por los correspondientes en español

In [23]:
nombres_dias_ingles = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
nombres_dias_espanol = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
df['DiaSemana'] = df['DiaSemana'].replace(nombres_dias_ingles, nombres_dias_espanol)

Agrego una nueva columna 'Mes' que contiene el nombre del mes correspondiente en español

In [24]:
df['Mes'] = df['release_date'].dt.month_name()

Reemplazo los nombres de los meses en inglés por los meses en español

In [25]:
nombres_meses_ingles = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
nombres_meses_espanol = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
df['Mes'] = df['Mes'].replace(nombres_meses_ingles, nombres_meses_espanol)

In [ ]:
df.head(5)

Elimino las columnas que no voy a utilizar

In [28]:
columns_to_drop = ['belongs_to_collection', 'original_language', 'overview', 'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'status', 'tagline']
df= df.drop(columns=columns_to_drop)

Compruebo que las columnas se hayan borrado

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   budget        45376 non-null  float64       
 1   genres        42992 non-null  object        
 2   id            45376 non-null  object        
 3   popularity    45376 non-null  object        
 4   release_date  45376 non-null  datetime64[ns]
 5   revenue       45376 non-null  float64       
 6   title         45376 non-null  object        
 7   vote_average  45376 non-null  float64       
 8   vote_count    45376 non-null  float64       
 9   release_year  45376 non-null  int64         
 10  return        45376 non-null  float64       
 11  DiaSemana     45376 non-null  object        
 12  Mes           45376 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(6)
memory usage: 4.8+ MB


Guardo el Dataset limpio

In [30]:
# Guardo

df.to_csv('df_movies_limpio.csv', index=False)

In [31]:
# Guardo en Google Drive nuestro Dataframe limpio
df.to_csv('/content/drive/MyDrive/df_movies_limpio.csv', index=False)